## This example contains sample script for feature processing and launching models 

In [1]:
import sys; sys.path.append("../")
import re
from pprint import pp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

from src.automl import AutoML

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-12-09 15:52:38]
/workspaces/python/venvs/lama_venv/lib/python3.9/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't install

In [2]:
RANDOM_STATE = 42
N_JOBS = 5
TARGET = "satisfaction"

### Data

In [3]:
df = pd.read_csv("../data/airlines_train.csv").drop(columns="Unnamed: 0").sample(n=10_000, random_state=RANDOM_STATE).assign(cnst=1); df.info()
X, y = df.drop(columns=TARGET), df[TARGET]

# rename X columns to remove "-" symbol (not processed by catboost)
X = X.rename(columns = lambda x:re.sub('-', '', x))
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test  = train_test_split(X, y, stratify=y, random_state=RANDOM_STATE, test_size=0.2)
X_test, X_infernece, y_test, y_infernece  = train_test_split(X_test, y_test, stratify=y_test, random_state=RANDOM_STATE, test_size=0.2)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_infernece = X_test.reset_index(drop=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 80638 to 16420
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 10000 non-null  int64  
 1   Gender                             10000 non-null  object 
 2   Customer Type                      10000 non-null  object 
 3   Age                                10000 non-null  int64  
 4   Type of Travel                     10000 non-null  object 
 5   Class                              10000 non-null  object 
 6   Flight Distance                    10000 non-null  int64  
 7   Inflight wifi service              10000 non-null  int64  
 8   Departure/Arrival time convenient  10000 non-null  int64  
 9   Ease of Online booking             10000 non-null  int64  
 10  Gate location                      10000 non-null  int64  
 11  Food and drink                     10000 non-null 

## AutoML

In [4]:
AutoML?

Init signature:
AutoML(
    task,
    use_preprocessing_pipeline=True,
    preprocessing_pipeline_kwargs={},
    use_val_test_pipeline=True,
    val_test_pipeline_kwargs={},
    feature_selector_type='CatboostByShap',
    feature_selector_kwargs={},
    auto_models_init_kwargs={},
    n_jobs=1,
    random_state=42,
    log_to_file=False,
)
Docstring:     
AutoML - automate routine ML tasks in 5 lines of code.

Parameters
----------
task
    Machine Learning task to solve, by default "classification"
    Should be one of:
        - "classification" (binary and multiclass)
        - "regression"
use_preprocessing_pipeline, optional
    Whether to use preprocessing pipeline, by default True
preprocessing_pipeline_kwargs, optional
    Keyword arguments to initialize preprocessing_pipeline, by default {}
    List of possible arguments and their default values:
        - pipe_steps = ['all']
        - nan_share_ts=0.2
        - qconst_feature_val_share_ts=0.95
        - impute_num_strategy='

In [5]:
AutoML.fit?

Signature:
AutoML.fit(
    self,
    X_train,
    y_train,
    X_test=None,
    y_test=None,
    max_obs_for_preproc=100000,
    auto_model_fit_kwargs={},
)
Docstring:
Fit the AutoML on train data.

Parameters
----------
X_train
    Pandas DataFrame with train features
y_train
    Pandas DataFrame or numpy array of train target values
X_test, optional
    Pandas DataFrame with test features, by default None
y_test, optional
    Pandas DataFrame or numpy array of test target values, by default None
max_obs_for_preproc, optional
    Maximum number of observations used to fit preprocessings, by default 100_000
    Smaller values speed up the execution, but may lead to inaccuracies.
auto_model_fit_kwargs, optional
    Keyword arguments to fit AutoModel, by default {}
    List of possible arguments and their default values:
        - tuning_timeout=60
        - save_models=False
        - save_params=True
        - save_oof=False
        - save_test=False
File:      /workspaces/python/proje

#### Init args

In [6]:
TASK = "classification"
N_JOBS = 5
LOG_TO_FILE = True

preprocessing_pipeline_kwargs = {"obj_encoders": ["oe"]}

feature_selector_type = "CatboostByShap"
feature_selector_kwargs = {"n_features_to_select": 10, "steps": 4}

auto_models_init_kwargs = {
    "metric": "roc_auc",
    "time_series": False,
    "models_list": ["linear", "forests", "boostings", "lama"],
    "blend": True,
    "stack": True
}

#### Fit args

In [7]:
auto_model_fit_kwargs = {
    "tuning_timeout": 60
}

In [8]:
automl = AutoML(
    task=TASK,
    preprocessing_pipeline_kwargs=preprocessing_pipeline_kwargs,
    feature_selector_type=feature_selector_type,
    feature_selector_kwargs=feature_selector_kwargs,
    auto_models_init_kwargs=auto_models_init_kwargs,
    n_jobs=N_JOBS,
    log_to_file=LOG_TO_FILE
)

[2024-12-09 15:52:39,339] - [  PREPROC   ] - Успешно заданы шаги pipeline
[2024-12-09 15:52:39,341] - [ VAL TESTS  ] - Успешно заданы шаги pipeline


In [11]:
automl.fit(X_train, y_train, X_test, y_test, auto_model_fit_kwargs=auto_model_fit_kwargs)

[2024-12-09 15:55:12,465] - [Pipeline] .. (step 1 of 6) Processing nan_cols_dropper, total=   0.0
[2024-12-09 15:55:12,506] - [Pipeline] ....... (step 2 of 6) Processing nan_imputer, total=   0.0
[2024-12-09 15:55:12,518] - [  PREPROC   ] - QConstant features to drop: ['cnst']
[2024-12-09 15:55:12,520] - [Pipeline] .... (step 3 of 6) Processing qconst_dropper, total=   0.0
[2024-12-09 15:55:12,530] - [  PREPROC   ] - Corr features to drop: ['Arrival Delay in Minutes', 'Arrival Delay in Minutes']
[2024-12-09 15:55:12,531] - [Pipeline] . (step 4 of 6) Processing corr_cols_dropper, total=   0.0
[2024-12-09 15:55:12,579] - [Pipeline] .... (step 5 of 6) Processing outlier_capper, total=   0.0
[2024-12-09 15:55:12,605] - [Pipeline] ... (step 6 of 6) Processing feature_encoder, total=   0.0
[2024-12-09 15:55:12,791] - [Pipeline] .......... (step 1 of 2) Processing PSI_test, total=   0.1
[2024-12-09 15:55:12,916] - [Pipeline] .. (step 2 of 2) Processing Adversarial_test, total=   0.1
[2024-12-